In [1]:
import json
import sys
import dropbox
import requests
import pandas as pd

import networkx as nx
import matplotlib
import pyvis

from pyvis.network import Network

In [2]:
ws_pathlist = json.loads(
    requests.get(
        "https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/whoscored-matchlist.json"
    ).content
)

In [3]:
dics = [
    json.loads(requests.get(ws_pathlist[i]).content) for i in range(len(ws_pathlist))
]

In [4]:
passes = []
for i in range(len(dics[2]["events"])):
    if (
        dics[2]["events"][i]["type"]["displayName"] == "Pass"
        and dics[2]["events"][i]["outcomeType"]["displayName"] == "Successful"
        and dics[2]["events"][i]["teamId"] == dics[2]["events"][i + 1]["teamId"]
    ):
        dics[2]["events"][i]["recieverId"] = dics[2]["events"][i + 1]["playerId"]
        passes.append(dics[2]["events"][i])

passes_df = pd.DataFrame(passes)

passes_df = passes_df[["playerId", "recieverId", "teamId"]]

passes_df["passes"] = "faszom"

test_df = passes_df.groupby(["playerId", "recieverId", "teamId"]).count().reset_index()

test_df = test_df[test_df["playerId"] != test_df["recieverId"]].reset_index(drop=True)

test_df["color"] = test_df.apply(
    lambda x: "red" if x["teamId"] == 67 else "blue", axis=1
)

In [5]:
players = []
for i in dics[2]["home"]["players"]:
    players.append([i["playerId"], i["name"]])
players = pd.DataFrame.from_records(players).rename(columns={0: "playerId", 1: "name"})
home_players = players.copy()

players = []
for i in dics[2]["away"]["players"]:
    players.append([i["playerId"], i["name"]])
players = pd.DataFrame.from_records(players).rename(columns={0: "playerId", 1: "name"})
away_players = players

players = pd.concat([home_players, away_players]).reset_index().drop("index", axis=1)

In [6]:
for i in range(len(test_df["playerId"])):
    test_df["playerId"][i] = (
        players["name"]
        .loc[players["playerId"] == test_df["playerId"][i]]
        .reset_index(drop=True)[0]
    )
    test_df["recieverId"][i] = (
        players["name"]
        .loc[players["playerId"] == test_df["recieverId"][i]]
        .reset_index(drop=True)[0]
    )

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stu

In [7]:
G=nx.DiGraph()
for i in range(1,(len(test_df))):
    G.add_edge(test_df.loc[i,'playerId'], test_df.loc[i,'recieverId'], weight=test_df.loc[i,'passes'])

cc=nx.closeness_centrality(G, u=None, distance=None, wf_improved=False)

CC_df=pd.DataFrame.from_dict(cc,orient='index', columns=['CC'])

test_df=test_df.merge(CC_df, how='left', left_on='playerId', right_index=True)

In [53]:
pass_net = Network(
    height="700",
    width="700",
    bgcolor="FFFFFF",
    font_color="black",
    directed=True,
    notebook=False,
)
pass_net.barnes_hut()
sources = test_df["playerId"]
targets = test_df["recieverId"]
weights = test_df["passes"]
size = test_df["CC"]
color = test_df["color"]

edge_data = zip(sources, targets, weights, size, color)
for e in edge_data:
    src = str(e[0])
    dst = str(e[1])
    w = e[2]
    s = e[3]
    c = e[4]
    pass_net.add_node(src, src, title=src, size=s, color=c)
    pass_net.add_node(dst, dst, title=dst, size=s, color=c)
    pass_net.add_edge(src, dst, value=w)
    neighbor_map = pass_net.get_adj_list()

for node in pass_net.nodes:
    node["title"] += " Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]])

pass_net.set_options(
    """
var options = {
  "nodes": {
    "borderWidth": 2,
    "color": {
      "highlight": {
        "background": "rgba(217,255,50,1)"
      }
    },
    "font": {
      "size": 50,
      "face": "tahoma"
    }
  },
  "edges": {
    "color": {
      "inherit": true
    },
    "smooth": false
  },
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -80000,
      "springLength": 250,
      "springConstant": 0.001
    },
    "minVelocity": 0.75
  }
}
"""
)

pass_net.show_buttons()
pass_net.show("pass_network.html")

AttributeError: 'dict' object has no attribute 'configure'

In [10]:
test_df

,playerId,recieverId,teamId,passes,color,CC
0,Jesús Navas,Fernando,67,4,red,0.764706
1,Jesús Navas,Éver Banega,67,7,red,0.764706
2,Jesús Navas,Luuk de Jong,67,2,red,0.764706
3,Jesús Navas,Tomás Vaclik,67,1,red,0.764706
4,Jesús Navas,Nemanja Gudelj,67,1,red,0.764706
...,...,...,...,...,...,...
186,Adrià Pedrosa,Jonathan Calleri,70,4,blue,0.666667
187,Adrià Pedrosa,Adrián Embarba,70,4,blue,0.666667
188,Adrià Pedrosa,Marc Roca,70,4,blue,0.666667
189,Víctor Gómez,Sergi Darder,70,1,blue,0.631579


In [11]:
players = []
for i in dics[2]["home"]["players"]:
    players.append([i["playerId"],i["name"]])
players = pd.DataFrame.from_records(players).rename(columns = {0:"playerId",1:"name"})
players["teamId"] = 67
home_players = players

In [12]:
players = []
for i in dics[2]["away"]["players"]:
    players.append([i["playerId"],i["name"]])
players = pd.DataFrame.from_records(players).rename(columns = {0:"playerId",1:"name"})
players["teamId"] = 70
away_players = players

In [13]:
home_players.head()

,playerId,name,teamId
0,68732,Tomás Vaclik,67
1,9446,Jesús Navas,67
2,134984,Diego Carlos,67
3,90778,Sergi Gómez,67
4,362275,Sergio Reguilón,67


In [14]:
away_players.head()

,playerId,name,teamId
0,10445,Diego López,70
1,80957,Javi López,70
2,67327,Bernardo Espinosa,70
3,73265,Leandro Cabrera,70
4,363966,Adrià Pedrosa,70


In [15]:
players = pd.concat([home_players,away_players])

In [16]:
draft = pd.merge(passes_df,players,how="left").rename(columns = {"name":"playerName"})

In [17]:
team_id_dic = {
   dics[2][v]["teamId"]: f"{v} - {dics[2][v]['name']}" for v in ["home", "away"]
}

In [18]:
team_id_dic

{67: 'home - Sevilla', 70: 'away - Espanyol'}

In [19]:
import dash_html_components as html

In [20]:
type(html.Img(src='pass_network.html'))

dash_html_components.Img.Img

In [21]:
html.Div([html.Iframe(srcDoc=open("/home/borza/Downloads/pass_network.html").read())])

FileNotFoundError: [Errno 2] No such file or directory: '/home/borza/Downloads/pass_network.html'

In [49]:
import dash_dangerously_set_inner_html
import dash
import dash_html_components as html

app = dash.Dash("Hello")


app.scripts.config.serve_locally = True
# plot = open("pass_network.html").read()

app.layout = html.Div(
    [html.Iframe(srcDoc=open("pass_network.html").read())], title="Pass Dash Graph")


if __name__ == "__main__":
    app.run_server(debug=True, use_reloader=False, port=8054)

Running on http://127.0.0.1:8054/
Running on http://127.0.0.1:8054/
Running on http://127.0.0.1:8054/
Running on http://127.0.0.1:8054/
Running on http://127.0.0.1:8054/
Running on http://127.0.0.1:8054/
Running on http://127.0.0.1:8054/
Running on http://127.0.0.1:8054/
Running on http://127.0.0.1:8054/
Debugger PIN: 471-422-197
Debugger PIN: 471-422-197
Debugger PIN: 471-422-197
Debugger PIN: 471-422-197
Debugger PIN: 471-422-197
Debugger PIN: 471-422-197
Debugger PIN: 471-422-197
Debugger PIN: 471-422-197
Debugger PIN: 471-422-197
 * Serving Flask app "Hello" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
